In [1]:
import sys

from google.protobuf.json_format import MessageToJson
from client_wrapper import ServiceClient

import nlpserv_pb2 as nlp_messages
import nlpserv_pb2_grpc as nlp_service
from utils import dump

client=ServiceClient(nlp_service, 'NlpProcsStub', 'localhost', 10052)

def tokenize(text):
    request = nlp_messages.NlTokenizerRequest(text=nlp_messages.NlText(text=text))
    response = client.Tokenizer(request)
    return response
def extract(text):
    request = nlp_messages.NlTokenizerRequest(text=nlp_messages.NlText(text=text))
    response = client.EntityExtractor(request)
    return response

# print(response)
response=tokenize("这里是北京")
for t in response.tokens:
    # print(MessageToJson(resp))
    print(t.text, t.label, t.length)

extract_r=extract("这里是北京")
for t in extract_r.entities:
    # print(MessageToJson(resp))
    print(t.entity, t.value, t.start, t.end)

这里 r 2
是 v 1
北京 ns 2
rzs 这里 0 2
vshi 是 2 3
ns 北京 3 5


In [4]:
text="十九元套餐包括什么"
request = nlp_messages.NlText(text=text)
response = client.ParseAmountTerms(request)
print(type(response))
for t in response.amount:
    print(t.numericVal, t.entity.start, t.entity.end)

<class 'nlpserv_pb2.NlAmountList'>
19 0 2


In [6]:
from rasa_nlu.tokenizers import Tokenizer, Token

def tokenize_msg(text, msg_tokens):         
    words=[]
    for token in msg_tokens.tokens:
        words.append(token.text)

    running_offset = 0
    tokens = []
    for word in words:
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        tokens.append(Token(word, word_offset))   
    return tokens

tokens=tokenize_msg("这里是北京", response)
for t in tokens:
    print(t.text, t.offset)

这里 0
是 2
北京 3


In [5]:
from rasa_nlu.tokenizers import Tokenizer, Token

def tokenize_msg(text, msg_tokens):         
    words=[]
    for token in msg_tokens.entities:
        words.append(token.value)

    running_offset = 0
    tokens = []
    for word in words:
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        tokens.append(Token(word, word_offset))   
    return tokens

text="这里是北京"
tokens=tokenize_msg(text, extract(text))
for t in tokens:
    print(t.text, t.offset)

这里 0
是 2
北京 3


In [13]:
def extract_entities(text, msg_tokens, filters):
    running_offset = 0
    entities = []
    for token in msg_tokens.tokens:
        word=token.text
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        if token.label in filters:
            entities.append({
                    "entity": token.label,
                    "value": token.text,
                    "start": word_offset,
                    "confidence": None,
                    "end": running_offset
                })
    return entities

extract_entities("这里是北京", response, ["ns", "nr"])

[{'confidence': None, 'end': 5, 'entity': 'ns', 'start': 3, 'value': '北京'}]

In [6]:
def extract_entities(text, msg_tokens, filters):
    running_offset = 0
    entities = []
    for token in msg_tokens.entities:     
        if token.entity in filters:
            entities.append({
                    "entity": token.entity,
                    "value": token.value,
                    "start": token.start,
                    "confidence": None,
                    "end": token.end
                })
    return entities

text="这里是北京"
extract_entities(text, extract(text), ["ns", "nr"])

[{'confidence': None, 'end': 5, 'entity': 'ns', 'start': 3, 'value': '北京'}]

In [9]:
# nz	其他专名: 除人名、国名、地名、团体、机构、组织以外的其他专有名词都标以nz。满族/nz，俄罗斯族/nz，汉语/nz，罗马利亚语/nz， 捷克语/nz，中文/nz， 英文/nz， 满人/nz， 哈萨克人/nz， 诺贝尔奖/nz， 茅盾奖/nz， 1.包含专有名称（或简称）的交通线，标以nz；短语型的，标为NZ。津浦路/nz， 石太线/nz， [京/j 九/j 铁路/n]NZ， [京/j 津/j 高速/b 公路/n]NZ， 2. 历史上重要事件、运动等专有名称一般是短语型的，按短语型专有名称处理，标以NZ。[卢沟桥/ns 事件/n]NZ， [西安/ns 事变/n]NZ，[五四/t 运动/n]NZ， [明治/nz 维新/n]NZ，[甲午/t 战争/n]NZ，3.专有名称后接多音节的名词，如“语言”、“文学”、“文化”、“方式”、“精神”等，失去专指性，则应分开。欧洲/ns 语言/n， 法国/ns 文学/n， 西方/ns 文化/n， 贝多芬/nr 交响乐/n， 雷锋/nr 精神/n， 美国/ns 方式/n，日本/ns 料理/n， 宋朝/t 古董/n 4. 商标（包括专名及后接的“牌”、“型”等）是专指的，标以nz，但其后所接的商品仍标以普通名词n。康师傅/nr 方便面/n， 中华牌/nz 香烟/n， 牡丹III型/nz 电视机/n， 联想/nz 电脑/n， 鳄鱼/nz 衬衣/n， 耐克/nz 鞋/n5. 以序号命名的名称一般不认为是专有名称。2/m 号/q 国道/n ，十一/m 届/q 三中全会/j如果前面有专名，合起来作为短语型专名。[中国/ns 101/m 国道/n]NZ， [中共/j 十一/m 届/q 三中全会/j]NZ，6. 书、报、杂志、文档、报告、协议、合同等的名称通常有书名号加以标识，不作为专有名词。由于这些名字往往较长，名字本身按常规处理。《/w 宁波/ns 日报/n 》/w ，《/w 鲁迅/nr 全集/n 》/w，中华/nz 读书/vn 报/n， 杜甫/nr 诗选/n，少数书名、报刊名等专有名称，则不切分。红楼梦/nz， 人民日报/nz，儒林外史/nz 7. 当有些专名无法分辨它们是人名还是地名或机构名时，暂标以nz。[巴黎/ns 贝尔希/nz 体育馆/n]NT，其中“贝尔希”只好暂标为nz。
mappings={"ns":"location", 
          "nr": "person", 
          "nt": "organization",
          "nz":"proper",
          "nx":"foreign",
          "s":"space",
          "t":"time",
          "mq":"amount",
          "m":"numeral",
          "nf":"food"
         }
def parse_print(text, filters):
    response=extract(text)
    result=extract_entities(text, response, filters)
    for en in result:
        print(mappings[en["entity"]], en.get("value"))

filters=mappings.keys()
parse_print("我的希望是希望张晚霞的背影被晚霞映红", ["ns", "nr", "nt", "s"])
parse_print("支援臺灣正體香港繁體：微软公司於1975年由比爾·蓋茲和保羅·艾倫創立。", filters)

person 张晚霞
location 香港
organization 微软公司
proper 於
amount 1975年
proper 艾


In [15]:
def tokenize_print(text):
    response=tokenize(text)
    for t in response.tokens:
        # print(MessageToJson(resp))
        print(t.text, t.label, t.length)

text="蓝翔给宁夏固原市彭阳县红河镇黑牛沟村捐赠了挖掘机"
parse_print(text, filters)
tokenize_print(text)

person 蓝翔
location 宁夏
location 固原市
location 彭阳县
location 红河镇
location 黑牛沟村
蓝翔 v 2
给 v 1
宁夏 ns 2
固原市 ns 3
彭阳县 ns 3
红河镇 ns 3
黑牛沟村 ns 4
捐赠 v 2
了 u 1
挖掘机 n 3


In [21]:
sents=[
        "十九元套餐包括什么",
        "九千九百九十九朵玫瑰",
        "壹佰块都不给我",
        "９０１２３４５６７８只蚂蚁",
        "牛奶三〇〇克*2",
        "ChinaJoy“扫黄”细则露胸超2厘米罚款",
        "我要买五瓶矿泉水"
]
for text in sents:
    parse_print(text, filters)
    print("~~~~")

amount 十九元
~~~~
amount 九千九百九十九朵
~~~~
amount 壹佰块
~~~~
amount ９０１２３４５６７８只
~~~~
organization 牛奶三〇〇克*
numeral 2
~~~~
foreign ChinaJoy
amount 2厘米
~~~~
proper 要买
numeral 五
food 矿泉水
~~~~


In [20]:
def extract_print(text):
    response=extract(text)
    for t in response.entities:
        # print(MessageToJson(resp))
        kind=t.entity
        if t.entity in mappings:
            kind=mappings.get(kind)
        print(kind, t.value, t.start)
for text in sents:
    extract_print(text)        
    print("~~~~")

amount 十九元 0
n 套餐 3
v 包括 5
ry 什么 7
~~~~
amount 九千九百九十九朵 0
n 玫瑰 8
~~~~
amount 壹佰块 0
d 都 3
d 不 4
p 给 5
rr 我 6
~~~~
amount ９０１２３４５６７８只 0
n 蚂蚁 11
~~~~
organization 牛奶三〇〇克* 0
numeral 2 7
~~~~
foreign ChinaJoy 0
w “ 8
vi 扫黄 9
w ” 11
n 细则 12
v 露 14
ng 胸 15
v 超 16
amount 2厘米 17
vi 罚款 20
~~~~
rr 我 0
proper 要买 1
numeral 五 3
n 瓶 4
food 矿泉水 5
~~~~
